# Prueba de bondad de ajuste

In [260]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from funciones import desagrupar_datos

## Distribucion discreta (Poisson)

Se dice que el numero de defectos en unas tarjetas madre sigue una distribucion Poisson. Se obtiene una muestra aleatoria de *n = 60* tarjetas madre y se observa el numero de defectos.

In [269]:
df = pd.DataFrame({'defectos': [0, 1, 2, 3], 'oi': [32, 15, 9, 4]})
df

,defectos,oi
0,0,32
1,1,15
2,2,9
3,3,4


Como no sabemos el promedio de la distribucion Poisson habra que estimarla a partir de los datos que tenemos. La estimacion del promedio de defectos por tarjeta madre es el promedio muestral, o sea, (32 * 0 + 15 * 1 + 9 * 2 + 4 * 3) / 60 = 0.75.

In [262]:
n = df['oi'].sum()
media = 0
for i, defectos in enumerate(df['defectos']):
	media += df['oi'][i] * defectos
media = media / n
media

0.75

O bien, podemos desagrupar los datos y obtener la media con una funcion

In [263]:
lista = df.to_records(index = False)
lista = list(lista)
defectos = desagrupar_datos(lista)
del lista
media = np.mean(defectos)
media

0.75

A partir de la distribucion Poisson con parametro 0.75, puede sacarse $p_{i}$ que es la probabilidad hipotetica asociada con el *i*-ésimo intervalo de clase. Como cada intervalo de clase corresponde a un numero particular de defectos, puede hallarse $p_{i}$ de la sig. manera:

$p_{1} = P(X = 0) = \frac{e^{-0.75}(0.75)^{0}}{0!} = 0.472$
<br>
$p_{2} = P(X = 1) = \frac{e^{-0.75}(0.75)^{1}}{1!} = 0.354$
<br>
$p_{3} = P(X = 2) = \frac{e^{-0.75}(0.75)^{2}}{2!} = 0.133$
<br>
$p_{4} = P(X \ge 3) = 1 - (p_{1} + p_{2} + p_{3}) = 0.041$

In [273]:
# df['p'] = df.apply(lambda x: (np.exp(-media) * (media ** x['defectos'])) / math.factorial(x['defectos']), axis = 1)
# df['p'][-1:] = 1 - (df['p'][0] + df['p'][1] + df['p'][2])
# df

p = []
for i in df['defectos']:
	print(i)
	if i < 3:
		p.append((np.exp(-media) * (media ** i)) / math.factorial(i))
	else:
		p.append(1 - sum(p))
print(p)

0
1
2
3
[0.4723665527410147, 0.354274914555761, 0.13285309295841038, 0.04050543974481391]


Las frecuencias esperadas se sacan multiplicando el tamaño de la muestra *n* por las probabilidades $p_{i}$. O sea, $E_{i} = np_{i}$

In [265]:
df['ei'] = df.apply(lambda i: n * i['p'], axis = 1)
df

,defectos,oi,p,ei
0,0,32,0.472367,28.341993
1,1,15,0.354275,21.256495
2,2,9,0.132853,7.971186
3,3,4,0.040505,2.430326


Como la frecuencia esperada de la ultima celda es menor que 3, se combinan las dos ultimas celdas.

In [266]:
df.groupby(['ei'], as_index = False).agg('sum')

,ei,defectos,oi,p
0,2.430326,3,4,0.040505
1,7.971186,2,9,0.132853
2,21.256495,1,15,0.354275
3,28.341993,0,32,0.472367


In [267]:
ultimos_dos = df.tail(2)
suma_ultimos_dos = ultimos_dos.sum()
suma_ultimos_dos = pd.DataFrame([suma_ultimos_dos])
suma_ultimos_dos['defectos'] = 2

In [268]:
df.drop(df.tail(2).index, inplace = True)
df = pd.concat([df, suma_ultimos_dos], ignore_index = True)
df

,defectos,oi,p,ei
0,0,32.0,0.472367,28.341993
1,1,15.0,0.354275,21.256495
2,2,13.0,0.173359,10.401512
